In [59]:
import os
import json
import pandas as pd
import zipfile
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# print(tf.config.list_physical_devices("GPU"))
data = pd.read_csv("../data/IMDB_reviews.csv")

data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [60]:
# data.shape


data["sentiment"].value_counts()


sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [61]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

data["sentiment"].value_counts()
# data.head()


/tmp/ipykernel_71941/3893172435.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


sentiment
1    25000
0    25000
Name: count, dtype: int64

In [62]:
train_data, test_data = train_test_split(
    data, random_state=42, shuffle=True, test_size=0.1
)


tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

y_train = train_data["sentiment"]
y_test = test_data["sentiment"]


In [66]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


/home/mrmauler/DRIVE/projects/dl/sentiment_analysis/sentiment_analysis_env/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [67]:
model.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [68]:
model.fit(X_train, y_train, epochs=3, batch_size=64, validation_split=0.2)

# loss = model.evaluate(X_test, y_test)


Epoch 1/3
563/563 ━━━━━━━━━━━━━━━━━━━━ 939s 2s/step - accuracy: 0.7383 - loss: 0.5123 - val_accuracy: 0.8573 - val_loss: 0.3438
Epoch 2/3
563/563 ━━━━━━━━━━━━━━━━━━━━ 987s 2s/step - accuracy: 0.8597 - loss: 0.3413 - val_accuracy: 0.8479 - val_loss: 0.3750
Epoch 3/3
563/563 ━━━━━━━━━━━━━━━━━━━━ 1009s 2s/step - accuracy: 0.8883 - loss: 0.2805 - val_accuracy: 0.8681 - val_loss: 0.3329
157/157 ━━━━━━━━━━━━━━━━━━━━ 58s 367ms/step - accuracy: 0.8714 - loss: 0.3267


In [70]:
loss, accuracy = model.evaluate(X_test, y_test)

print(f"Loss :{loss}")
print(f"Accuracy :{accuracy}")


157/157 ━━━━━━━━━━━━━━━━━━━━ 47s 297ms/step - accuracy: 0.8714 - loss: 0.3267
Loss :0.3158178925514221
Accuracy :0.8766000270843506


In [1]:
def predict_sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    # padded_sequence = pad_sequences(tokenizer.texts_to_sequences(review), maxlen=200)
    prediction = model.predict(padded_sequence)

    sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
    return sentiment


In [75]:
rev = "that movie was great"

print(predict_sentiment(rev))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
positive
